<a href="https://colab.research.google.com/github/nbpyth97/Exercise/blob/master/tp1/ps2022_tp1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Processamento de Streams 2022
## TP1 - Air Quality Monitoring (airborne particulate matter)
-- version April 6 
 - updated to full dataset.

-- version April 8 
 - added code for spark streaming (unstructured)

-- version April 12
 - added a note to highlight the unstructured data
 format has the timestamp at the last position.




The goal of this project is to analyze data provided by a set of air quality sensors [sds011](https://aqicn.org/sensor/sds011/pt/). The sensors present in the dataset are located in Portugal, namely in the Lisbon metro area. Each sensor provides two values: measuring particles less than 10 µm (P1) and less than 2.5 µm (P2) in μg/m³.

The sensor data, spans the first half of 2020, and is streamed of Kafka. 

Each data sample has the following schema:

sensor_id | sensor_type | location | latitude | longitude | timestamp | P1 | P2
----------|-------------|----------|----------|-----------|-----------|----|---
string  | string | string | float | float| timestamp | float | float



## Questions


1. Find the time of day with the poorest air quality, for each location. Updated daily;
2. Find the average air quality, for each location. Updated hourly;
3. Can you show any daily and/or weekly patterns to air quality?;
4. The data covers a period of extensive population confinement due to Covid 19. Can you find a signal in the data showing air quality improvement coinciding with the confinement period?

## Requeriments

1. Solve each question using one of the systems studied in the course.
2. For questions not fully specified, provide your own interpretation, given your own analysis of the data.

## Grading Criteria 

1. Bonus marks will be given for solving questions using more than one system (eg. Spark Unstructured + Spark Structured);
2. Bonus marks will be given if some kind of graphical output is provided to present the results;
3. Grading will also take into account the general clarity of the programming and of the presentation report (notebook).




### Deadline

30th April + 1 day - ***no penalty***

For each day late, ***0.5 / day penalty***. Penalty accumulates until
the grade of the assignment reaches 8.0.

---
### Colab Setup


In [2]:
#@title Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#@title Install PySpark
!pip install pyspark findspark --quiet
import findspark
findspark.init()
findspark.find()

     |████████████████████████████████| 281.4 MB 35 kB/s 
     |████████████████████████████████| 198 kB 52.2 MB/s 


'/usr/local/lib/python3.7/dist-packages/pyspark'

In [3]:
#@title Install & Launch Kafka
%%bash
KAFKA_VERSION=3.1.0
KAFKA=kafka_2.13-$KAFKA_VERSION
wget -q -O /tmp/$KAFKA.tgz https://dlcdn.apache.org/kafka/$KAFKA_VERSION/$KAFKA.tgz
tar xfz /tmp/$KAFKA.tgz
wget -q -O $KAFKA/config/server1.properties - https://github.com/smduarte/ps2022/raw/main/colab/server1.properties

UUID=`$KAFKA/bin/kafka-storage.sh random-uuid`
$KAFKA/bin/kafka-storage.sh format -t $UUID -c $KAFKA/config/server1.properties
$KAFKA/bin/kafka-server-start.sh -daemon $KAFKA/config/server1.properties


Formatting /tmp/kraft-combined-logs


### Air quality sensor data publisher
This a small python Kafka client that publishes a continous stream of text lines, obtained from the periodic output of the sensors.

* The Kafka server is accessible @localhost:9092 
* The events are published to the `air_quality` topic
* Events are published 3600x faster than realtime relative to the timestamp


In [4]:
#@title Start Kafka Publisher
%%bash
pip install kafka-python dataclasses --quiet
wget -q -O - https://github.com/smduarte/ps2022/raw/main/colab/kafka-tp1-logsender.tgz | tar xfz - 2> /dev/null

cd kafka-tp1-logsender
nohup python publisher.py --filename 2020-01-06_sds011-pt.csv --topic air_quality  --speedup 3600 2> publisher-error.log > publisher-out.log &

The python code below shows the basics needed to process JSON data from Kafka source using PySpark.

Spark Streaming python documentation is found [here](https://spark.apache.org/docs/latest/api/python/reference/pyspark.streaming.html)

---
#### PySpark Kafka Stream Example


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

def dumpBatchDF(df, epoch_id):
    df.show(20, False)

spark = SparkSession \
    .builder \
    .appName('Kafka Spark Structured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1') \
    .getOrCreate()

lines = spark \
  .readStream \
  .format('kafka') \
  .option('kafka.bootstrap.servers', 'localhost:9092') \
  .option('subscribe', 'air_quality') \
  .option('startingOffsets', 'earliest') \
  .load() \
  .selectExpr('CAST(value AS STRING)')


schema = StructType([StructField('timestamp', TimestampType(), True),
                     StructField('sensor_id', StringType(), True),
                     StructField('sensor_type', StringType(), True),
                     StructField('location', StringType(), True),
                     StructField('latitude', FloatType(), True),
                     StructField('longitude', FloatType(), True),
                     StructField('p1', FloatType(), True),
                     StructField('p2', FloatType(), True)])

lines = lines.select( from_json(col('value'), schema).alias('data')).select('data.*')

query = lines \
    .writeStream \
    .outputMode('append') \
    .foreachBatch(dumpBatchDF) \
    .start()

query.awaitTermination(600)
query.stop()
spark.stop()

A saída de streaming foi truncada nas últimas 5000 linhas.
|timestamp          |sensor_id|sensor_type|location|latitude|longitude|p1   |p2   |
+-------------------+---------+-----------+--------+--------+---------+-----+-----+
|2020-01-15 20:07:06|25874    |SDS011     |13691   |37.09194|-8.683101|28.98|7.43 |
|2020-01-15 20:07:43|33204    |SDS011     |19563   |39.53239|-8.92895 |23.73|6.32 |
|2020-01-15 20:07:45|27393    |SDS011     |14858   |41.2    |-8.326   |10.65|3.55 |
|2020-01-15 20:08:03|4638     |SDS011     |2332    |38.646  |-9.154   |46.6 |12.23|
|2020-01-15 20:08:43|20000    |SDS011     |10161   |41.188  |-8.642   |65.7 |13.9 |
|2020-01-15 20:09:32|25874    |SDS011     |13691   |37.09194|-8.683101|31.05|7.47 |
|2020-01-15 20:10:09|33204    |SDS011     |19563   |39.53239|-8.92895 |27.92|8.45 |
|2020-01-15 20:10:13|27393    |SDS011     |14858   |41.2    |-8.326   |9.02 |3.37 |
|2020-01-15 20:10:29|4638     |SDS011     |2332    |38.646  |-9.154   |53.3 |11.93|
|2020-01-15 20:11

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
RuntimeError: reentrant call inside <_io.BufferedReader name=57>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 504, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/py4j/clientserver.py", line 475, in send_command
    answer = smart_decode(self.stream.readline()[:-1

+-------------------+---------+-----------+--------+--------+---------+------+-----+
|timestamp          |sensor_id|sensor_type|location|latitude|longitude|p1    |p2   |
+-------------------+---------+-----------+--------+--------+---------+------+-----+
|2020-01-17 10:17:39|27393    |SDS011     |14858   |41.2    |-8.326   |16.6  |8.52 |
|2020-01-17 10:17:49|25874    |SDS011     |13691   |37.09194|-8.683101|18.95 |9.9  |
|2020-01-17 10:18:13|20000    |SDS011     |10161   |41.188  |-8.642   |10.07 |3.17 |
|2020-01-17 10:18:51|4638     |SDS011     |2332    |38.646  |-9.154   |113.97|8.2  |
|2020-01-17 10:18:51|33204    |SDS011     |19563   |39.53239|-8.92895 |22.4  |4.07 |
|2020-01-17 10:20:08|27393    |SDS011     |14858   |41.2    |-8.326   |18.75 |8.05 |
|2020-01-17 10:20:15|25874    |SDS011     |13691   |37.09194|-8.683101|19.83 |10.43|
|2020-01-17 10:20:38|20000    |SDS011     |10161   |41.188  |-8.642   |15.3  |3.4  |
|2020-01-17 10:21:17|33204    |SDS011     |19563   |39.53239|-8.9

Py4JError: ignored

+-------------------+---------+-----------+--------+--------+---------+-----+-----+
|timestamp          |sensor_id|sensor_type|location|latitude|longitude|p1   |p2   |
+-------------------+---------+-----------+--------+--------+---------+-----+-----+
|2020-01-17 10:40:44|33204    |SDS011     |19563   |39.53239|-8.92895 |10.93|2.03 |
|2020-01-17 10:40:44|4638     |SDS011     |2332    |38.646  |-9.154   |24.63|13.9 |
|2020-01-17 10:42:10|25874    |SDS011     |13691   |37.09194|-8.683101|18.5 |6.37 |
|2020-01-17 10:42:28|27393    |SDS011     |14858   |41.2    |-8.326   |18.98|9.9  |
|2020-01-17 10:42:31|20000    |SDS011     |10161   |41.188  |-8.642   |13.9 |3.5  |
|2020-01-17 10:43:10|4638     |SDS011     |2332    |38.646  |-9.154   |20.87|11.7 |
|2020-01-17 10:43:10|33204    |SDS011     |19563   |39.53239|-8.92895 |7.47 |2.15 |
|2020-01-17 10:44:36|25874    |SDS011     |13691   |37.09194|-8.683101|20.48|6.45 |
|2020-01-17 10:44:56|20000    |SDS011     |10161   |41.188  |-8.642   |13.67

### Spark Streaming (UnStructured) 

Latest Spark does not support Kafka sources with UnStructured Streaming.

The next cell publishes the dataset using a TCP server, running at port 7777. For this mode, there is no need to install or run Kafka, using the cell above.

The events are played faster than "realtime", at a 3600x speedup, such that 1 hour in terms of dataset timestamps is
sent in 1 second realtime, provided the machine is fast enough. As such, Spark Streaming window functions need to be sized accordingly, since a minibatch of 1 second will be
worth 1 hour of dataset events.

In [4]:
%%bash

git clone https://github.com/smduarte/ps2022.git 2> /dev/null > /dev/null || git -C ps2022 pull
cd ps2022/colab/socket-tp1-logsender/

nohup python publisher.py --filename 2020-01-06_sds011-pt.csv --port 7777  --speedup 3600 2> /tmp/publisher-error.log > /tmp/publisher-out.log &

Each line sample has the following parts separated by blanks:

sensor_id | sensor_type | location | latitude | longitude | P1 | P2 | timestamp 
----------|-------------|----------|----------|-----------|-----------|----|---
string  | string | string | float | float| float | float | timestamp



In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

import matplotlib.pyplot as plt
import pandas

spark = SparkSession \
    .builder \
    .appName('Kafka Spark UnStructured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1') \
    .getOrCreate()

try:
  ssc = StreamingContext(spark.sparkContext,1)
  lines = ssc.socketTextStream('localhost', 7777)
  line = lines.window(1,24)

  results=line.filter(lambda x : len(x)>0)\
  .map(lambda x:((x.split(' ')[2]+' '+x.split(' ')[-1]),x.split(' ')[-3]))\
  .reduceByKey(lambda a,b: max(a,b))

  results.pprint()
    
  ssc.start()
  ssc.awaitTermination(75)
except Exception as err:
  print(err)
ssc.stop()
spark.stop()



Find the average air quality, for each location. Updated hourly;

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

spark = SparkSession \
    .builder \
    .appName('Kafka Spark UnStructured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1') \
    .getOrCreate()

try:
  ssc = StreamingContext(spark.sparkContext, 1)
  lines = ssc.socketTextStream('localhost', 7777) 

  results=lines.filter(lambda x : len(x)>0)\
  .map(lambda x:((x.split(' ')[2]+' '+x.split(' ')[-1][11:13]),(float(x.split(' ')[-3]),1)))\
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
  .map(lambda x : (x[0],x[1][0]/x[1][1]))

  results.pprint()
    
  ssc.start()
  ssc.awaitTermination(5)
except Exception as err:
  print(err)
ssc.stop()
spark.stop()

Can you show any daily and/or weekly patterns to air quality?

Each line sample has the following parts separated by blanks:

sensor_id | sensor_type | location | latitude | longitude | P1 | P2 | timestamp 
----------|-------------|----------|----------|-----------|-----------|----|---
string  | string | string | float | float| float | float | timestamp

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode, split, count
import pandas
import matplotlib.pyplot as plt


spark = SparkSession \
    .builder \
    .appName('StructuredWebLogExample') \
    .getOrCreate()

lines = spark.readStream.format('socket') \
    .option('host', 'localhost') \
    .option('port', 7777) \
    .load()
  
sl = split( lines.value, ' ')
table = lines \
      .withColumn('sensor_id', sl.getItem(0).cast('string'))\
      .withColumn('sensor_type', sl.getItem(1).cast('string'))\
      .withColumn('location', sl.getItem(2).cast('string'))\
      .withColumn('latitude', sl.getItem(3).cast('float'))\
      .withColumn('longitude', sl.getItem(4).cast('float'))\
      .withColumn('P1', sl.getItem(5).cast('float'))\
      .withColumn('P2', sl.getItem(6).cast('float'))\
      .withColumn('timestamp', sl.getItem(7).cast('timestamp'))\
      .drop('value')
try:

  results = table \
            .groupBy('location') \
            .count()

except Exception as err:
  print(err)

'''
x = results.select('timestamp').toList()
y = results.select('P2').toList()
plt.bar(x,y)
plt.show()
'''

query = results \
    .writeStream \
    .outputMode('complete') \
    .trigger(processingTime='1 seconds') \
    .foreachBatch(lambda df, epoch: df.show(10, False)) \
    .start()

query.awaitTermination(5)
query.stop()

In [6]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext

spark = SparkSession \
    .builder \
    .appName('Kafka Spark UnStructured Streaming Example') \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1') \
    .getOrCreate()

try:
  ssc = StreamingContext(spark.sparkContext, 1)
  lines = ssc.socketTextStream('localhost', 7777) 

  results=lines.filter(lambda x : len(x)>0)\
  .map(lambda x:((x.split(' ')[2]+' '+x.split(' ')[-1][11:13]),(float(x.split(' ')[-3]),1)))\
  .reduceByKey(lambda x, y: (x[0]+y[0], x[1]+y[1]))\
  .map(lambda x : (x[0],x[1][0]/x[1][1])).dumpBatchDF()
    
  ssc.start()
  ssc.awaitTermination(5)
except Exception as err:
  print(err)
ssc.stop()
spark.stop()

'TransformedDStream' object has no attribute 'dumpBatchDF'
